# Training

In [9]:
# Loading useful libraries

import numpy as np
import pandas as pd

import optuna
import os
import pickle
import random

from lightgbm import LGBMClassifier, LGBMRegressor
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, r2_score
from sklearn.model_selection import cross_val_score

In [2]:
# Selecting "regression" or "classification"

model_type = "classification"

assert(model_type in ["regression", "classification"])

In [3]:
# Loading data

X_train = pd.read_csv(f'../tmp/X_train_{model_type}.csv', index_col='User_ID')
y_train = pd.read_csv(f'../tmp/y_train_{model_type}.csv', index_col='User_ID')['is_BigSpender']

In [4]:
def objective(trial):
    # Defining the hyperparameters to search over
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 5,1000),
        'num_leaves': trial.suggest_int('max_depth', 10, 50),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
    }

    # Evaluating the Random Forest model with the current set of hyperparameters
    if model_type == "regression":
        model = LGBMRegressor(**params, n_jobs=-1, random_state=42)
        metric = r2_score
    else:
        model = LGBMClassifier(**params, n_jobs=-1, random_state=42)
        metric = recall_score
    
    random.seed(42)

    train_idx = random.sample(list(X_train.index), round(X_train.shape[0] * 4/5))
    valid_idx = list(set(X_train.index) - set(train_idx))

    model.fit(X_train.loc[train_idx], y_train.loc[train_idx], )
    y_pred = model.predict(X_train.loc[valid_idx])
    score_valid = metric(y_train.loc[valid_idx], y_pred)
    print(score_valid)
    return score_valid

# Creating a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Printing the best hyperparameters and corresponding accuracy
print('Best Trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print(f'    {key}: {value}')

[I 2023-11-29 15:26:13,869] A new study created in memory with name: no-name-5515d770-48ac-4d4d-930c-1baad4a6e810
c:\Users\enric\anaconda3\envs\gcp_demo2\Lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "max_depth"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 50}
  warnings.warn(


[LightGBM] [Info] Number of positive: 1124, number of negative: 2646
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 3770, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.298143 -> initscore=-0.856155
[LightGBM] [Info] Start training from score -0.856155


[I 2023-11-29 15:26:14,577] Trial 0 finished with value: 0.3931034482758621 and parameters: {'n_estimators': 743, 'max_depth': 46}. Best is trial 0 with value: 0.3931034482758621.


0.3931034482758621
[LightGBM] [Info] Number of positive: 1124, number of negative: 2646
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000057 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 3770, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.298143 -> initscore=-0.856155
[LightGBM] [Info] Start training from score -0.856155


c:\Users\enric\anaconda3\envs\gcp_demo2\Lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "max_depth"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 50}
  warnings.warn(
[I 2023-11-29 15:26:15,219] Trial 1 finished with value: 0.38275862068965516 and parameters: {'n_estimators': 763, 'max_depth': 49}. Best is trial 0 with value: 0.3931034482758621.


...

0.4
[LightGBM] [Info] Number of positive: 1124, number of negative: 2646
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 3770, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.298143 -> initscore=-0.856155
[LightGBM] [Info] Start training from score -0.856155


c:\Users\enric\anaconda3\envs\gcp_demo2\Lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "max_depth"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 50}
  warnings.warn(
[I 2023-11-29 15:27:03,912] Trial 98 finished with value: 0.3896551724137931 and parameters: {'n_estimators': 704, 'max_depth': 34}. Best is trial 92 with value: 0.40344827586206894.


0.3896551724137931
[LightGBM] [Info] Number of positive: 1124, number of negative: 2646
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 3770, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.298143 -> initscore=-0.856155
[LightGBM] [Info] Start training from score -0.856155


c:\Users\enric\anaconda3\envs\gcp_demo2\Lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "max_depth"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 50}
  warnings.warn(
[I 2023-11-29 15:27:04,467] Trial 99 finished with value: 0.3896551724137931 and parameters: {'n_estimators': 751, 'max_depth': 37}. Best is trial 92 with value: 0.40344827586206894.


0.3896551724137931
Best Trial:
  Value:  0.40344827586206894
  Params: 
    n_estimators: 808
    max_depth: 34


In [5]:
seed = 42

if model_type == "regression":
    model = LGBMRegressor(**study.best_trial.params, n_jobs=-1, random_state=seed)
    metric = r2_score
else:
    model = LGBMClassifier(**study.best_trial.params, n_jobs=-1, random_state=seed)
    metric = recall_score
# model = LGBMClassifier(n_estimators=10000, n_jobs=-1, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
score_train = metric(y_train, y_pred)
score_train

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1414, number of negative: 3298
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 4712, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.300085 -> initscore=-0.846894
[LightGBM] [Info] Start training from score -0.846894
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


0.5862800565770863

In [6]:
metric.__name__.split('_')[0]

'recall'

In [7]:
scores = cross_val_score(model, X_train, y_train, cv=5, scoring=metric.__name__.split('_')[0])
print(scores)
scores.mean()

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1131, number of negative: 2638
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 3769, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.300080 -> initscore=-0.846919
[LightGBM] [Info] Start training from score -0.846919
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't 

0.3507580883642834

# Saving model

In [ ]:
try:
   os.makedirs("../tmp")
except FileExistsError:
   # directory already exists
   pass

In [13]:
with open('../tmp/model.pkl', 'wb') as f:
    pickle.dump(model, f)